<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/experiment/c620_jax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/skywalker0803r/c620.git
!pip install autorch > log.txt

fatal: destination path 'c620' already exists and is not an empty directory.


In [2]:
from c620.F import F
from c620.config import config
import joblib
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import os
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
from jax.numpy import linalg
from numpy import nanargmin,nanargmax 
key = random.PRNGKey(42)

os.chdir('/content/c620')
print(os.getcwd())
!git pull

/content/c620
Already up to date.


In [3]:
def sp2wt(x,s):
  a = 100*x*s
  b = np.diag(x@s.T).reshape(-1,1)
  return a/b

def j(s,s_):
  return np.mean(np.square(s-s_))

def g(s_):
  return np.sum(np.array([np.square((np.sum(s_[:,[i,i+41,i+41*2,i+41*3]])-1)) for i in range(41)])) # mass balance

def h(x,s_,Benzene_in_Sidedraw):
  wt_ = sp2wt(x,s_)[0]
  return np.mean(np.square(wt_[7]-Benzene_in_Sidedraw)) # Benzene_in_Sidedraw condition

In [4]:
import pandas as pd
import joblib
df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0)
idx = df.sample(1).index
c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
sp = df.filter(regex='Split Factors Calculation_Split Factor for Individual Component').loc[idx]
feed_col = c['x41']
feed = df[feed_col].loc[idx]
wt_col = c['vent_gas_x']+c['distillate_x']+c['sidedraw_x']+c['bottoms_x']
wt = df[wt_col].loc[idx]
case = df.loc[idx,c['case']]
Benzene_in_Sidedraw = df.loc[idx,c['case'][2]].to_frame()
print(c['sidedraw_x'].index('Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'))

7


# forward test

In [5]:
f = F(config)
f.Recommended_mode = False
f.real_data_mode = False

In [6]:
c620_input = case.join(feed)
sp_ = f.c620_model.predict(c620_input).iloc[:,:41*4]
print(j(sp.values,sp_.values))
print(g(sp.values))
print(h(feed.values,sp.values[:,41*2:41*3],Benzene_in_Sidedraw.values))

1.6338147e-06
9.450218e-13
0.0


In [7]:
def Lagrange(l): 
  return j(sp.values,l[[0],0:164])- l[0,165]*g(l[[0],0:164]) - l[0,166]*h(feed.values,l[[0],41*2:41*3],Benzene_in_Sidedraw.values)
L = jacfwd(Lagrange)
def solveLagrangian(l): 
  return l - 0.1*L(l)  

In [8]:
domain = random.uniform(key, shape=(1,166), dtype='float32',minval=0.0, maxval=1.0)
for epoch in tqdm(range(150)):
  domain = solveLagrangian(domain)

In [14]:
minimums = j(sp.values,domain[[0],0:164])
arglist = nanargmin(minimums)
argmin = domain[arglist]
minimum = minimums[arglist]

ValueError: ignored